**Model Specification**
- Model 1 – Non-Hierarchical Survival Model

    - Likelihood: Exponential (or Weibull)

    - Linear predictor:

        - log(λ_i) = α + β1*age_i + β2*drug_i + β3*sex_i + β4*serBilir_i + ...


    - Priors: α, β ~ Normal(0,1)

    - Goal: estimate hazard ratios and effect sizes

In [ ]:
# ============================================================
# Bayesian Survival Models
# ============================================================

import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import pytensor.tensor as pt

# ------------------------------------------------------------
# Load baseline dataset
# ------------------------------------------------------------

df = pd.read_csv("data/processed/pbc_clean.csv")

df_baseline = (
    df.sort_values(["id", "year"])
      .groupby("id", as_index=False)
      .first()
)

# Survival data
T = df_baseline["years"].values
E = df_baseline["status2"].values  # 1 = event, 0 = censored

# Separate continuous and categorical covariates
continuous_vars = ["age", "serBilir", "albumin"]
categorical_vars = ["sex", "drug", "edema"]

# Scale continuous covariates
scaler = StandardScaler()
X_cont = scaler.fit_transform(df_baseline[continuous_vars])

# Keep categorical covariates as-is
X_cat = df_baseline[categorical_vars].values

# Combine design matrix
X = np.column_stack([X_cont, X_cat])

N, P = X.shape


In [ ]:
# ============================================================
# Model 1: Non-Hierarchical Exponential Survival Model
# ============================================================

with pm.Model() as model1:

    alpha = pm.Normal("alpha", mu=0, sigma=1)
    beta = pm.Normal("beta", mu=0, sigma=1, shape=P)

    log_lambda = alpha + pm.math.dot(X, beta)
    lambda_ = pm.math.exp(log_lambda)

    # FAST & STABLE likelihood
    loglik = (
        E * (pm.math.log(lambda_) - lambda_ * T) +
        (1 - E) * (-lambda_ * T)
    )

    pm.Potential("likelihood", loglik.sum())
    #pm.Deterministic("log_likelihood", loglik)

    trace_model1 = pm.sample(
        draws=2000,
        tune=1000,
        chains=4,
        target_accept=0.9,
        return_inferencedata=True
    )

# Save results
az.to_netcdf(trace_model1, "results/models/model1_new_trace.nc")
